In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from metrics import dice_loss, dice_coef, iou

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from model import build_unet

H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def load_data(dataset_path):
    images = sorted(glob(os.path.join(dataset_path, 'dermoscopic_image', "*.png")))
    return images

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x                             

def save_results(ori_x, y_pred, save_image_path):
    y_pred = np.expand_dims(y_pred, axis=-1)  
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    cat_images = np.concatenate([y_pred*255], axis=1)
    cv2.imwrite(save_image_path, cat_images)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving results """
    create_dir("hair_mask")
    
    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("files/model.h5")

    dataset_path = '...'#provide path to the dataset parent folder with dermoscopic images in dermoscopic_image subfolder
    images = load_data(dataset_path)

    for x in images:
        #exctract image name
        name = x.split("/")[-1]
        print(name)

        #read the image and mask
        ori_x, x = read_image(x)

        #predicting the mask
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        #saving the predicted mask
        save_image_path = f"hair_mask/{name}"
        save_results(ori_x, y_pred, save_image_path)